```{currentmodule} optimap
```

In [ ]:
from optimap.utils import jupyter_render_animation as render

```{tip}
Download this tutorial as a {download}`Jupyter notebook <converted/phase.ipynb>`, or a {download}`python script <converted/phase.py>` with code cells. We highly recommend using [Visual Studio Code](#vscode) to execute this tutorial.
```

# Tutorial 9: Phase and Singularities during VF

In this tutorial, we will learn how to compute phase maps of action potential vortex waves during ventricular fibrillation (VF). Using the phase maps, we will compute and track phase singularities across the epicardial surface of an isolated intact rabbit heart during VF. Phase singularities (PS) indicate rotational core regions of reentrant vortex waves (spiral waves) and can be used to characterize the complexity of VF or to document interactions of reentrant waves with the substrate. For example, a higher number of PS can indicate more aggresive episodes of VF, moving PS are associated with meandering spiral waves and high motility, and PS can attach to scar tissue and other heterogeneities. `optimap` provides all routines which are necessary to compute and filter phase videos, and calculate and track phase singularities. Similar analysis was performed in {footcite:t}`Christoph2018,Lebert2021` among many other publications.

Here we use the VF example file from [Tutorial 1](basics.ipynb) and compensate the residual contractile motion to work with a video without motion. We then compute a pixel-wise normalized motion-stabilized video and mask out the background to focus on the electrical dynamics on the epicardial surface.

In [ ]:
import optimap as om
import numpy as np

filepath = om.download_example_data('VF_Rabbit_1.npy')
video = om.load_video(filepath, frames=500)
video_warped = om.motion_compensate(video,
                                    contrast_kernel=5,
                                    presmooth_spatial=1,
                                    presmooth_temporal=1)

video_warped_normalized = om.video.normalize_pixelwise_slidingwindow(video_warped, window_size=60)

mask = om.background_mask(video_warped[0], show=False)
video_warped_normalized[:, mask] = np.nan

The pixel-wise normalized motion-stabilized video displays action potential vortex waves as black waves (when using a black-and-white colormap):

In [ ]:
om.show_video(video_warped_normalized, interval=40);

In [ ]:
render(lambda: om.show_video(video_warped_normalized, interval=40))

Next, we will first compute and display a phase video from the pixel-wise normalized video. Alternatively to a pixel-wise normalized video one could also compute phase maps from a frame-wise differentiated video, see below. We will then smooth the phase video using spatio-temporal phase smoothing filters, and ,lastly, calculate and track phase singularities through the video.

## Computing Phase Videos

Phase videos can be computed from normalized video data in which the optical signals fluctuate between [0,1]. Because we already normalized the video above, `optimap`'s {func}`compute_phase` function can compute the phase automatically:

In [ ]:
video_phase = om.phase.compute_phase(video_warped_normalized)

The phase angle was computed for each time-series in the video individually using the Hilbert transform. All values in the resulting phase video lie on the interval [$-\pi$, $\pi$]. Now, let's have a look at the phase video:

In [ ]:
om.show_video(video_phase, title='phase video', cmap='hsv', interval=40);

In [ ]:
render(lambda: om.show_video(video_phase, title='phase video', cmap='hsv', interval=40))

We chose the continuous colormap 'hsv' which displays both $-\pi$ and $\pi$ in red. The 'hsv' colormap has the following advantages: 1) there is no discontinuity at the phase shift from $-\pi$ to $\pi$, 2) all lines of equal phase are displayed by lines of equal color, 3) it is easier to identify pinwheel patterns or 'rotors'. 

The phase video contains a lot of noise as it was derived from the raw pixel-wise normalized video. In order to remove the noise in the phase video, we will now apply different phase filters which convert the phase video data into complex phase data and operate on this complex data:

$$
\begin{align}
    \phi(t, x, y) \rightarrow \cos{\phi}(t,x,y) + i \cdot \sin{\phi}(t,x,y)
\end{align}
$$

First, we will use a phase coherence filter {func}`phasefilter_angle_threshold`, which will detect and remove outliers in the phase video by computing the complex order parameter $r$ for each pixel and removing pixels with low $r$ (defined by threshold parameter, typically 0.8 - 0.9).The complex order parameter $r$ is calculated by summing the complex vectors describing the phase of each pixel on the unit circle within a small disc-shaped region around the pixel and computing their cumulative magnitude:

$$
\begin{align}
    r \cdot e^{\phi} = \frac{1}{N} \sum \limits_i^N e^{i\phi_j}
\end{align}
$$

where $j = 1,...,N$ is the number of complex phase values within a spatio-temporal kernel around each pixel with diameter $2 \dot wx + 1$ pixels and a temporal window size of $2 \cdot wt + 1$ video frames, see also {footcite:t}`Lebert2021`.

In [ ]:
video_phase_filtered = om.phase.phasefilter_angle_threshold(video_phase, wx=3, wy=3, wt=1, tr_angle=0.9)

In the resulting filtered phase video all outliers were replaced with NaNs, which are being displayed as white pixels:

In [ ]:
om.show_video(video_phase_filtered, title='removed outliers', cmap='hsv', interval=40);

In [ ]:
render(lambda: om.show_video(video_phase_filtered, title='removed outliers', cmap='hsv', interval=40))

Now we can infill all NaN pixels with interpolated phase values from the surrounding tissue, which we compute using the {func}`phasefilter_fillsmooth` function that operates on the complex phase values:

```{warning}
This tutorial is currently work in progress. We will add more information soon.
```

Further reading:

```{footbibliography}
```